Importing the Dataset.

In [2]:
import pandas as pd
import numpy as np

# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)

import tensorflow as tf
from tensorflow.keras import layers

In [3]:
#Importing the CSV Dataset.
users_data = pd.read_csv(r'C:\Users\moyo\OneDrive\Documents\Codes\Web\Node\e-learning\ml\datasets\courses.csv', delimiter=';')
users_data.head()
users_data.shape

FileNotFoundError: ignored